In [ ]:
%pip install linearmodels

In [ ]:
import pandas as pd
df = pd.read_csv('/content/drive/MyDrive/EMBABE/final.csv')

In [ ]:
df.shape

(243692, 40)

In [ ]:
df = df[df['amountPaidOnBuildingClaim']<1500000]
df = df[df['buildingDamageAmount']<2000000]
df = df[df['BuildingAge']<150]
df = df[df['amountPaidOnIncreasedCostOfComplianceClaim']<32000]
df = df[df['buildingPropertyValue']!=0]

In [ ]:
df.shape

(243498, 46)

In [ ]:
import numpy as np

# Check the available columns in the dataframe
print(df.columns)

log_transform_cols = [
    'policyCount',
    'totalBuildingInsuranceCoverage',
    'buildingPropertyValue',
    'buildingReplacementCost', # Corrected column name
    'waterDepth',
]

for col in log_transform_cols:
    # Add a small constant to handle zero values before taking the log
    df['log' + col] = np.log(df[col] + 1)

# For waterDepth, use signed log transformation as it can be negative
df['signedLogWwaterDepth'] = np.sign(df['waterDepth']) * np.log(np.abs(df['waterDepth']) + 1)

display(df.head())

Index(['agricultureStructureIndicator', 'policyCount',
       'elevatedBuildingIndicator', 'houseWorship', 'locationOfContents',
       'numberOfFloorsInTheInsuredBuilding', 'nonProfitIndicator',
       'obstructionType', 'occupancyType', 'amountPaidOnBuildingClaim',
       'amountPaidOnContentsClaim',
       'amountPaidOnIncreasedCostOfComplianceClaim',
       'postFIRMConstructionIndicator', 'smallBusinessIndicatorBuilding',
       'totalBuildingInsuranceCoverage', 'totalContentsInsuranceCoverage',
       'yearOfLoss', 'primaryResidenceIndicator', 'buildingDamageAmount',
       'buildingPropertyValue', 'disasterAssistanceCoverageRequired',
       'ficoNumber', 'floodWaterDuration', 'floodproofedIndicator',
       'iccCoverage', 'netIccPaymentAmount', 'nfipRatedCommunityNumber',
       'numberOfUnits', 'buildingReplacementCost', 'replacementCostBasis',
       'stateOwnedIndicator', 'waterDepth', 'rentalPropertyIndicator',
       'reportedZipCode', 'countyCode', 'censusTract', 'censusB

/usr/local/lib/python3.11/dist-packages/pandas/core/arraylike.py:399: RuntimeWarning: divide by zero encountered in log
  result = getattr(ufunc, method)(*inputs, **kwargs)
/usr/local/lib/python3.11/dist-packages/pandas/core/arraylike.py:399: RuntimeWarning: invalid value encountered in log
  result = getattr(ufunc, method)(*inputs, **kwargs)


,agricultureStructureIndicator,policyCount,elevatedBuildingIndicator,houseWorship,locationOfContents,numberOfFloorsInTheInsuredBuilding,nonProfitIndicator,obstructionType,occupancyType,amountPaidOnBuildingClaim,...,censusBlockGroupFips,latitude,longitude,BuildingAge,logpolicyCount,logtotalBuildingInsuranceCoverage,logbuildingPropertyValue,logbuildingReplacementCost,logwaterDepth,signedLogWwaterDepth
0,0,1,1,0,3,1,0,10,1,27213.28,...,4.820120e+11,29.9,-95.4,39.649555,0.693147,12.429220,11.637741,11.719753,0.693147,0.693147
1,0,1,0,0,4,2,0,10,2,10004.72,...,3.712900e+11,34.1,-77.9,28.153320,0.693147,11.480413,11.508546,11.647815,0.000000,0.000000
2,0,1,0,0,4,2,0,10,1,17054.76,...,3.403120e+11,40.9,-74.2,36.254620,0.693147,10.755794,11.429413,11.627860,0.693147,0.693147
3,0,1,1,0,3,1,0,10,4,10800.00,...,2.211400e+11,29.9,-92.1,23.334702,0.693147,10.499601,11.595565,12.011079,1.791759,1.791759
4,0,1,0,0,3,1,0,10,1,14286.10,...,1.202700e+11,27.2,-82.0,9.483915,0.693147,11.130215,10.628884,0.000000,1.386294,1.386294


In [ ]:
import statsmodels.api as sm

# Define the independent variables (X) and the dependent variable (y)
X_cols = [  'logpolicyCount',
 'postFIRMConstructionIndicator',
 'smallBusinessIndicatorBuilding',
 'logtotalBuildingInsuranceCoverage',
 'totalContentsInsuranceCoverage',
#  'yearOfLoss',
 'primaryResidenceIndicator',
 'buildingDamageAmount',
#  'logbuildingPropertyValue',
 'iccCoverage',
 'logbuildingReplacementCost',
 'signedLogWwaterDepth',
 'rentalPropertyIndicator',
#  'BuildingAge',
]

y = df['amountPaidOnBuildingClaim']
X = df[X_cols]

# Add a constant to the independent variables
X = sm.add_constant(X)

# Fit the OLS model
model = sm.OLS(y, X).fit()

# Print the OLS summary
print(model.summary())

                                OLS Regression Results                               
Dep. Variable:     amountPaidOnBuildingClaim   R-squared:                       0.871
Model:                                   OLS   Adj. R-squared:                  0.871
Method:                        Least Squares   F-statistic:                 1.493e+05
Date:                       Thu, 19 Jun 2025   Prob (F-statistic):               0.00
Time:                               12:34:20   Log-Likelihood:            -2.7335e+06
No. Observations:                     243498   AIC:                         5.467e+06
Df Residuals:                         243486   BIC:                         5.467e+06
Df Model:                                 11                                         
Covariance Type:                   nonrobust                                         
                                        coef    std err          t      P>|t|      [0.025      0.975]
--------------------------------------

In [ ]:
# Perform the Ramsey RESET test
reset_test = sm.stats.diagnostic.linear_reset(model)

# Print the test results
print(reset_test)

<Wald test (chi2): statistic=75238.11315022569, p-value=0.0, df_denom=2>


## First IV combination

In [ ]:
from linearmodels.iv import IV2SLS
import statsmodels.api as sm

# Define the endogenous variable, instrumental variables, and other exogenous variables
endog = df['amountPaidOnBuildingClaim']
exog = sm.add_constant(df[['logpolicyCount',
                           'postFIRMConstructionIndicator',
                           'totalContentsInsuranceCoverage',
                           'primaryResidenceIndicator',
                           'buildingDamageAmount',
                           'logbuildingPropertyValue',
                           'iccCoverage',
                           'logbuildingReplacementCost',
                           'signedLogWwaterDepth',
                           'rentalPropertyIndicator',
                           'BuildingAge']])
# Exogenous variables which are also included in the original regression
endog_variable = df['logtotalBuildingInsuranceCoverage']
instrumental_variables = df[['smallBusinessIndicatorBuilding', 'yearOfLoss']]

# Fit the 2SLS model
tsls_model = IV2SLS(endog, exog, endog_variable, instrumental_variables).fit(cov_type='unadjusted')

# Print the 2SLS summary
print(tsls_model.summary)

# Perform the Wu-Hausman test
hausman_test = tsls_model.wu_hausman()

# Print the test results
print(hausman_test)

hansen_j = tsls_model.sargan
print("\nHansen J Test (Sargan Test):")
print(hansen_j)

                              IV-2SLS Estimation Summary                             
Dep. Variable:     amountPaidOnBuildingClaim   R-squared:                      0.5217
Estimator:                           IV-2SLS   Adj. R-squared:                 0.5217
No. Observations:                     243498   F-statistic:                 4.426e+05
Date:                       Thu, Jun 19 2025   P-value (F-stat)                0.0000
Time:                               16:17:20   Distribution:                 chi2(12)
Cov. Estimator:                   unadjusted                                         
                                                                                     
                                         Parameter Estimates                                         
                                   Parameter  Std. Err.     T-stat    P-value    Lower CI    Upper CI
-----------------------------------------------------------------------------------------------------
const 

The Hansen J test for overidentification yields a test statistic of 423.64 (p-value ~ 0.000). Thus, we reject the null hypothesis of instrument exogeneity. Rejecting the null hypothesis means that there is strong statistical evidence to suggest that at least one of instrumental variables is not valid. In other words, it is likely correlated with the error term in your structural equation.

## Second IV Combination

In [ ]:
from linearmodels.iv import IV2SLS
import statsmodels.api as sm

# Define the endogenous variable, instrumental variables, and other exogenous variables
endog = df['amountPaidOnBuildingClaim']
# Remove 'logbuildingPropertyValue' from exog when it's the endogenous variable
exog_cols = ['logpolicyCount',
             'postFIRMConstructionIndicator',
             'totalContentsInsuranceCoverage',
             'primaryResidenceIndicator',
             'buildingDamageAmount',
             'iccCoverage',
             'logbuildingReplacementCost',
             'signedLogWwaterDepth',
             'rentalPropertyIndicator',
             ]
exog = sm.add_constant(df[exog_cols])

endog_variable = df['logbuildingPropertyValue']
instrumental_variables = df[['BuildingAge', 'yearOfLoss']]

# Fit the 2SLS model
tsls_model = IV2SLS(endog, exog, endog_variable, instrumental_variables).fit()

# Print the 2SLS summary
print(tsls_model.summary)

# Perform the Wu-Hausman test
hausman_test = tsls_model.wu_hausman()

# Print the test results
print(hausman_test)

                              IV-2SLS Estimation Summary                             
Dep. Variable:     amountPaidOnBuildingClaim   R-squared:                      0.8009
Estimator:                           IV-2SLS   Adj. R-squared:                 0.8009
No. Observations:                     243498   F-statistic:                 1.722e+05
Date:                       Thu, Jun 19 2025   P-value (F-stat)                0.0000
Time:                               12:34:46   Distribution:                 chi2(10)
Cov. Estimator:                       robust                                         
                                                                                     
                                       Parameter Estimates                                        
                                Parameter  Std. Err.     T-stat    P-value    Lower CI    Upper CI
--------------------------------------------------------------------------------------------------
const          

We used a two-stage least squares (2SLS) regression to estimate the effect of logbuildingPropertyValue on amountPaidOnBuildingClaim, using BuildingAge and yearOfLoss as instrumental variables. The Wu-Hausman test statistic was 2053.1074 (p < 0.001), strongly rejecting the null hypothesis of exogeneity. This confirms that logbuildingPropertyValue is endogenous in our setting, and that OLS estimates would be inconsistent. Therefore, IV/2SLS results are required for unbiased inference.

## Weak Instrument Test

In [ ]:
# Print first-stage summary (for F-statistic)
print(tsls_model.first_stage)


              First Stage Estimation Results             
                                 logbuildingPropertyValue
---------------------------------------------------------
R-squared                                          0.3468
Partial R-squared                                  0.0160
Shea's R-squared                                   0.0160
Partial F-statistic                                2445.4
P-value (Partial F-stat)                           0.0000
Partial F-stat Distn                              chi2(2)
================================             ============
const                                             -25.651
                                                (-32.924)
logpolicyCount                                     0.9448
                                                 (68.579)
postFIRMConstructionIndicator                      0.2077
                                                 (51.824)
totalContentsInsuranceCoverage                  6.111e-06
              

The first-stage regression for logbuildingPropertyValue yields a partial F-statistic of 2445.4 (p < 0.001), far exceeding the conventional threshold of 10. This confirms that the instruments (BuildingAge, yearOfLoss) are highly relevant and not weak. Thus, you can be confident in the strength of your instruments for the IV/2SLS estimation.

In [ ]:
hansen_j = tsls_model.sargan
print("\nHansen J Test (Sargan Test):")
print(hansen_j)


Hansen J Test (Sargan Test):
Sargan's test of overidentification
H0: The model is not overidentified.
Statistic: 0.7646
P-value: 0.3819
Distributed: chi2(1)


The Hansen J test for overidentification yields a test statistic of 0.76 (p-value = 0.38). Thus, we fail to reject the null hypothesis of instrument exogeneity, supporting the validity of the instruments used in the IV estimation.